# LinkedIn Profile Scrapper

## Intro

Lets first create a bot before we start downoading profile-data. To be able to connect to linked via a bot selenium and beatifulsoup4 will be used. 

In [6]:
!pip install selenium


### Lets import the modules need for this project

In [33]:
import os, random, sys, time
from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys



## Create the web-driver of Firefox

In [94]:
driver = webdriver.Firefox()
driver = webdriver.Firefox()
driver.get("https://www.linkedin.com/uas/login")


### Login to LinkedIn

After inspecting the login page of LinkedIn, the following recon is gathered:

1. Username box has the ID username
2. Password box has the ID password
3. Login button has the class btn

Firstly we will assigned out LinkedIn credentials into un en pw variables. Secondly we will open the login page and enter our credentials and login to our profile. Lastly we will search for a random profile.



In [95]:
# The credentials are stored in config.txt in the project folder. 
# first row, [0], contains username and the second row, [1], contains password

creds = open('config.txt') # Opening the txt file
lines = creds.readlines() # extracting lines into lines var

un = lines[0] # Username
pw = lines[1] # Password

In [96]:
elementID = driver.find_element_by_id('username') # Finding the username box
elementID.send_keys(un) # entering our username 



In [97]:
elementID = driver.find_element_by_id('password') # Finding the password box
elementID.send_keys(pw) # entering our password

In [98]:
elementID.submit() # Clicking the login button

Now we are logged into LinkedIn with our credentials. Lets search for a random profile:

In [99]:
profile_link = '/in/romée-de-vries-6515a6173/' # this the links the profile
full_link = 'https://www.linkedin.com'+profile_link # this is the path before the profile specific part of the URL
driver.get(full_link)

## Get user ID's (profile_link) from the page source by using Beautifsoup

As shown below, BS4 is using the our Firefox driver to download the page source

In [100]:
BeautifulSoup(driver.page_source)

<html class="theme theme--mercado artdeco" lang="nl"><head>
<script type="application/javascript">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);</script>
<title>Romée de Vries | LinkedIn</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta class="mercado-icons-sprite" content="https://static-exp1.licdn.com/sc/h/3ki0iq40x0w1ti93600pfmk84" id="artdeco-icons/static/images/sprite-asset" name="asset-url"/>
<meta content="" name="description"/>
<meta content="notranslate" name="google"/>
<meta content="true" name="is-http2"/>
<meta content="voyager-web" name="service"/>
<meta content="#0077B5" name="theme-color"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<meta content="https://static-exp1.licdn.com/sc/h" name="baseCDNUrl"/>
<

### Functions to search the page source, get the user profile link

The page source contains basically the all html elements and data. We will into a the html section "People also viewed/Ook bekeken". So this sections and the name of the section is frequently updated by LinkedIn. So if the names of html elements change, this code will not work (So update the code, frequently)

Steps: 
- Creating an empty list for already checked profiles
- Creating an empty list for queued profiles
- Define a function to search the page source, find the section for the given class element
- In the section/class find all a tags where each a represents a user profile in the pages section Ook bekeken
- Scan/loop the a tags: add into queuedProfiles if the current a not in visited AND queued lists. 

In [102]:
visitedProfiles = []
queuedProfiles = []

def getNewProfileIDs(soup, queuedProfiles):
    profileID = []
    pav = soup.find('section', {'class': 'artdeco-card pv-browsemap-section__wrapper ember-view'})
    all_links = pav.findAll('a', {'class': "ember-view pv-browsemap-section__member align-items-center"})
    
    for link in all_links:
        userID = link.get('href')
        
        if ((userID not in queuedProfiles) and (userID not in visitedProfiles)):
            
            profileID.append(userID)
    return profileID

In [103]:
getNewProfileIDs(BeautifulSoup(driver.page_source), queuedProfiles)

['/in/mariam-a-9a7b11194/',
 '/in/bernadett-babis-08aa6a154/',
 '/in/katharina-mackenthun/',
 '/in/esthervandermaten/',
 '/in/postleon/']

Now it is time to fill out queuedProfiles list with userIDs/links by using our function

In [109]:
queuedProfiles = getNewProfileIDs(BeautifulSoup(driver.page_source), queuedProfiles)

In [110]:
queuedProfiles

['/in/rom%C3%A9e-de-vries-6515a6173/',
 '/in/laila-aldreibe-819a95193/',
 '/in/nguyenductrinhan/',
 '/in/vivian-karume-838023160/',
 '/in/melvin-dreves-9295b8179/']

## Now lets make something fun and make connections to random people up to 10.000 profile

We will get the last item in our queuedProfiles list and open it with the driver. After that we will inspect the elements to find send message button and send a standard message. In each opened profile we will apply our functiongetNewProfileID's to gather more UserID's up to 10.000 and loop in the same procedure. 

In [ ]:
while queuedProfiles:
    visitingProfileID = queuedProfiles.pop() # Get the last item in from the list
    visitedProfiles.append(visitingProfileID) # add the item into visited so that we dont send our message to same profiel
    full_link = "https://www.linkedin.com" + visitingProfileID # make full url link for the drive
    
    driver.get(full_link) # Open de full link
    
    soup = BeautifulSoup(driver.page_source) # get page source code
    try:
        queuedProfiles.extend(getNewProfileIDs(soup, queuedProfiles)) # apply the function, get new users ID's
    except:
        print('continue')
    

**For now the code is running forever: it opens a profile, gets more links to profiles that are in the section People also viewed. Adds into a list and walks trhough the same procedure**